In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [65]:
ee.Initialize()

daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
     .filter(ee.Filter.date('2011-01-01', '2011-12-31')) \
     .mean() ##Will mess up the toBands() function if mean is included - see below (daymet2) for this operation
##Note: there are 7 measurements per day (2555 per year except on leapyears) 

daymet2 = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
     .filter(ee.Filter.date('2000-01-01', '2000-12-31'))

daymet3 = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
     .geometry = ee.Geometry.Polygon(
        [
            [
                [-81.1641, 31.5554],
                [-81.2959, 31.2036],
                [-81.7464, 31.2835],
                [-81.7464, 31.6817]
            ]
        ]
    )


dm_precip = daymet.select('prcp')
dm_maxtemp = daymet.select('tmax')
dm_mintemp = daymet.select('tmin')
dm_daylight = daymet.select('dayl')
dm_srad = daymet.select('srad')
dm_snow = daymet.select('swe')
dm_vapor = daymet.select('vp')

prcpVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

daylVis = {
  'min': 0,
  'max': 50000.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

sradVis = {
  'min': 0,
  'max': 400.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

vpVis = {
  'min': 0,
  'max': 1500,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = geemap.Map(center=[40,-100], zoom=4)
Map.addLayer(dm_precip, prcpVis, "Precipitation")
Map.addLayer(dm_maxtemp, prcpVis, "Maxtemp")
Map.addLayer(dm_mintemp, prcpVis, "Mintemp")
Map.addLayer(dm_daylight, daylVis, "Daylight")
Map.addLayer(dm_srad, sradVis, "SRAD")
Map.addLayer(dm_snow, prcpVis, "Snowpack water?")
Map.addLayer(dm_vapor, vpVis, "Vaporpressure")

Map

##For the year 2000, site near blackbeard creek entrance is missing daymet data! Check to see if that's the same for all years
##EDIT: it looks like it's going to be for all years

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [20]:
daymet.getInfo()
daymet2.getInfo()
# daymet3.getInfo()

{'type': 'Polygon',
 'coordinates': [[[-81.1641, 31.5554],
   [-81.7464, 31.6817],
   [-81.7464, 31.2835],
   [-81.2959, 31.2036],
   [-81.1641, 31.5554]]]}

In [4]:
##Convert imagecollection to bands/ee.image, from: 
##https://ecodata.nrel.colostate.edu/gdpe-gee-remote-sensing-lessons/module4-interoperability.html#daySmet-weather-data

DaymetImage = daymet2.toBands()

##check this site out to try to average bands for analysis: 
##https://gis.stackexchange.com/questions/384421/temporally-reducing-image-bands-using-google-earth-engine

# Mean_dm = daymet \
#   .toBands() \
#   .filterBounds(geometry)

# print(DaymetImage, "DaymetImage")

In [67]:
##Exporting pixel values from points
coords_year = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Daymet/GA/Coords_year/NoFlower/coords_2011.csv'
##NoFlower folder is identical to Coords_year folder, with flowering status removed (confuses csv_to_ee function)
fc_coords = geemap.csv_to_ee(coords_year, latitude = "Latitude", longitude = "Longitude")

Map.addLayer(fc_coords, {}, "fc_coords")

##export data
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'daymet2011.csv')

# geemap.extract_values_to_points(fc_coords, daymet, out_csv, scale = 500) #Scale specified due to error.MAKE SURE it's right

##note: scale = 500 was chosen because 1000 yielded more NaNs. Same results as scale = 1000

##Depricated:
# geemap.extract_values_to_points(fc_coords, DaymetImage, out_csv)
#out_shp = os.path.join(out_dir, 'points.shp') ##shapefile exporting

In [23]:
DaymetImage.getInfo()

{'type': 'Image',
 'bands': [{'id': '20000101_dayl',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [9612, 8075],
   'crs': 'PROJCS["unnamed", \n  GEOGCS["NAD83", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Lambert_Conformal_Conic_2SP"], \n  PARAMETER["central_meridian", -100.0], \n  PARAMETER["latitude_of_origin", 42.5], \n  PARAMETER["standard_parallel_1", 60.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["scale_factor", 1.0], \n  PARAMETER["standard_parallel_2", 25.0], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]',
   'crs_transform': [1000, 0, -5802750, 0, -1000, 4984500]},
  {'id': '20000101_prcp',
   'data_type': {'type': 'PixelType', 'preci